In [1]:
import numpy as np
import HandTrackingModule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import logging
import cv2

# Setup logging
logging.basicConfig(filename="app.log", level=logging.INFO, format='%(asctime)s - %(message)s')

def initialize_volume():
    try:
        devices = AudioUtilities.GetSpeakers()
        interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        volume = interface.QueryInterface(IAudioEndpointVolume)
        vol_range = volume.GetVolumeRange()
        return volume, vol_range
    except Exception as e:
        logging.error(f"Failed to initialize volume: {e}")
        raise

def main():
    try:
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            logging.error("Could not open camera.")
            raise Exception("Could not open camera.")

        volume, vol_range = initialize_volume()
        detector = htm.handDetector(detectionCon=0.8, trackCon=0.7)
        minVol, maxVol = vol_range[0], vol_range[1]
        prevVol = 0

        # Further compacted canvas size
        canvas_width, canvas_height = 120, 240

        while True:
            success, frame = cap.read()
            if not success:
                logging.warning("Failed to read frame from camera.")
                continue

            # Create canvas with modern gradient
            canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
            for i in range(canvas_height):
                color = (0, i/canvas_height*100, i/canvas_height*255)
                cv2.line(canvas, (0, i), (canvas_width, i), color, 1)

            hands_img = detector.findHands(frame)
            lmList = detector.findPosition(hands_img, draw=False)

            if lmList:
                x1, y1 = lmList[4][1], lmList[4][2]
                x2, y2 = lmList[8][1], lmList[8][2]
                length = math.hypot(x2 - x1, y2 - y1)

                vol = np.interp(length, [30, 230], [minVol, maxVol])
                volBar = np.interp(length, [30, 220], [canvas_height-32, 72])
                volPer = np.interp(length, [30, 220], [0, 100])

                # More responsive smoothing
                smoothedVol = prevVol * 0.7 + vol * 0.3
                volume.SetMasterVolumeLevel(smoothedVol, None)
                prevVol = smoothedVol

                # Draw volume level with green gradient
                for i in range(int(volBar), canvas_height-32):
                    color = (0, 255 - (i-canvas_height+32)/208*255, 0)
                    cv2.line(canvas, (20, i), (canvas_width-20, i), color, 1)
                
                # Display volume percentage
                cv2.putText(canvas, f'{int(volPer)}%', (canvas_width//2 - 24, canvas_height-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2, cv2.LINE_AA)

            cv2.imshow('Volume Control', canvas)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    except Exception as e:
        logging.error(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

# Verision 2.0

In [ ]:
import numpy as np
import HandTrackingModule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import logging
import cv2

# Setup logging
logging.basicConfig(filename="app.log", level=logging.INFO, format='%(asctime)s - %(message)s')

def linear_to_decibel(linear):
    """Convert a linear scale to decibels."""
    if linear == 0:
        return -65.25  # Min volume
    min_dB_scale = np.power(10, -65.25 / 20)
    return 20 * np.log10(linear * (1 - min_dB_scale) + min_dB_scale)

def initialize_volume():
    try:
        devices = AudioUtilities.GetSpeakers()
        interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        volume = interface.QueryInterface(IAudioEndpointVolume)
        vol_range = volume.GetVolumeRange()
        return volume, vol_range
    except Exception as e:
        logging.error(f"Failed to initialize volume: {e}")
        raise

def main():
    try:
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            logging.error("Could not open camera.")
            raise Exception("Could not open camera.")

        volume, vol_range = initialize_volume()
        detector = htm.handDetector(detectionCon=0.8, trackCon=0.5)
        minVol, maxVol = vol_range[0], vol_range[1]

        # Further compacted canvas size
        canvas_width, canvas_height = 120, 240

        while True:
            success, frame = cap.read()
            if not success:
                logging.warning("Failed to read frame from camera.")
                continue

            # Create canvas with modern gradient
            canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
            for i in range(canvas_height):
                color = (0, i/canvas_height*100, i/canvas_height*255)
                cv2.line(canvas, (0, i), (canvas_width, i), color, 1)

            hands_img = detector.findHands(frame)
            lmList = detector.findPosition(hands_img, draw=False)

            if lmList:
                x1, y1 = lmList[4][1], lmList[4][2]
                x2, y2 = lmList[8][1], lmList[8][2]
                length = math.hypot(x2 - x1, y2 - y1)

                volPer = np.interp(length, [30, 230], [0, 100])
                vol_dB = linear_to_decibel(volPer / 100)
                vol = np.interp(vol_dB, [minVol, maxVol], [minVol, maxVol])
                volBar = np.interp(volPer, [0, 100], [canvas_height-32, 72])

                volume.SetMasterVolumeLevel(vol, None)

                # Draw volume level with green gradient
                for i in range(int(volBar), canvas_height-32):
                    color = (0, 255 - (i-canvas_height+32)/208*255, 0)
                    cv2.line(canvas, (20, i), (canvas_width-20, i), color, 1)
                
                # Display volume percentage
                cv2.putText(canvas, f'{int(volPer)}%', (canvas_width//2 - 24, canvas_height-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2, cv2.LINE_AA)

            cv2.imshow('Volume Control', canvas)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    except Exception as e:
        logging.error(f"An error occurred: {e}")

if __name__ == "__main__":
    main()